<a href="https://colab.research.google.com/github/sincereQK/QK-ML-Data-study/blob/main/stock_predict_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 파일은 Kaggle에 있는 📊Stock Market Analysis 📈 + Prediction using LSTM을 보면서 번역하고 공부하기위해 작성한 것입니다.

원본 파일 : https://www.kaggle.com/code/faressayah/stock-market-analysis-prediction-using-lstm

# 데이터 프로젝트 - 주식 시장 분석


시계열 데이터는 시간 순서대로 인덱싱된 일련의 데이터 포인트입니다. 시계열 데이터는 어디에나 존재하므로, 이를 다루는 것은 모든 데이터 분석가나 데이터 과학자에게 중요합니다.

이 노트북에서는 주식 시장 데이터, 특히 일부 기술주(Apple, Amazon, Google, Microsoft)를 발견하고 탐색할 것입니다.
yfinance를 사용하여 주식 정보를 얻고, Seaborn과 Matplotlib을 사용하여 다양한 측면을 시각화하는 방법을 배울 것입니다.
주식의 과거 성과 이력을 바탕으로 주식의 리스크를 분석하는 몇 가지 방법을 살펴보겠습니다. 또한, 장단기 메모리(LSTM) 방법을 통해 미래 주가를 예측해 볼 것입니다!

진행 과정에서 다음 질문에 답해볼 것입니다:

    1.) 시간에 따른 주가의 변화는 어떠했는가?
    2.) 주식의 평균 일일 수익률은 얼마였는가?
    3.) 여러 주식의 이동 평균은 어떠했는가?
    4.) 다른 주식 간의 상관 관계는 무엇이었는가?
    5.) 특정 주식에 투자함으로써 감수해야 할 리스크 가치는 얼마인가?
    6.) 미래 주가 움직임을 어떻게 예측할 수 있는가? (LSTM을 사용하여 APPLE 주식의 종가 예측)

***

## 데이터 가져오기

첫 번째 단계는 데이터를 가져와 메모리에 로드하는 것입니다.
주식 데이터는 Yahoo Finance 웹사이트에서 가져옵니다.
Yahoo Finance는 금융 시장 데이터와 매력적인 투자를 찾기 위한 도구가 풍부한 자료입니다.
Yahoo Finance에서 데이터를 가져오기 위해 yfinance 라이브러리를 사용할 것이며, 이 라이브러리는 Yahoo에서 시장 데이터를 다운로드하는 스레드 방식의 파이썬다운 방법을 제공합니다.
yfinance에 대해 더 알아보려면 이 글을 확인하세요: [Reliably download historical market data from with Python](https://aroussi.com/post/python-yahoo-finance)

# 1. 시간에 따른 주가 변화는 어떠했는가?

이 섹션에서는 pandas로 주식 정보를 요청하고 주식의 기본 속성을 분석하는 방법을 살펴보겠습니다.

In [ ]:
pip install -q yfinance

기존의 코드를 사용시
AttributeError: module 'yfinance' has no attribute 'pdr_override' 라는 오류가 발생

찾아보니 다음과 같은 해결방법을 제시
https://github.com/ranaroussi/yfinance/issues/2001
https://github.com/ranaroussi/yfinance/issues/2001#issuecomment-2485344382


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
from pandas_datareader.data import DataReader
import yfinance as yf

# yfinance 0.2.41버전부터 pdr_override() 기능이 삭제됨
# yf.pdr_override()

# For time stamps
from datetime import datetime

ticker = 'AAPL'
end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)
print(end)
print(start)

# AAPL = yf.download(stock, start, end)

df = yf.download(ticker,start,end,auto_adjust=False)
print(df)

#df = pd.Dataframe(AAPL,axis=0)
#df.tail(10)


2025-08-31 08:03:31.545221
2024-08-31 00:00:00


[*********************100%***********************]  1 of 1 completed

Price        Adj Close       Close        High         Low        Open  \
Ticker            AAPL        AAPL        AAPL        AAPL        AAPL   
Date                                                                     
2024-09-03  221.738037  222.770004  229.000000  221.169998  228.550003   
2024-09-04  219.826920  220.850006  221.779999  217.479996  221.660004   
2024-09-05  221.349838  222.380005  225.479996  221.520004  221.630005   
2024-09-06  219.797058  220.820007  225.240005  219.770004  223.949997   
2024-09-09  219.886642  220.910004  221.270004  216.710007  220.820007   
...                ...         ...         ...         ...         ...   
2025-08-25  227.160004  227.160004  229.300003  226.229996  226.479996   
2025-08-26  229.309998  229.309998  229.490005  224.690002  226.869995   
2025-08-27  230.490005  230.490005  230.899994  228.259995  228.610001   
2025-08-28  232.559998  232.559998  233.410004  229.339996  230.820007   
2025-08-29  232.139999  232.139999  23

회사 한개의 데이터는 잘 불러오지만 for문을 이용하여 여러개를 불러올떄 제대로 불러오지 못하고 있음

In [ ]:
# 하드 코딩으로 진행
apple_stock  = yf.download('AAPL', start, end, auto_adjust=False)
print(apple_stock.tail(3))

GOOGLE_stock  = yf.download('GOOG', start, end, auto_adjust=False)
print(GOOGLE_stock.tail(3))

MICROSOFT_stock  = yf.download('MSFT', start, end, auto_adjust=False)
print(MICROSOFT_stock.tail(3))

AMAZON_stock  = yf.download('AMZN', start, end, auto_adjust=False)
AMAZON_stock.tail(3)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Price        Adj Close       Close        High         Low        Open  \
Ticker            AAPL        AAPL        AAPL        AAPL        AAPL   
Date                                                                     
2025-08-27  230.490005  230.490005  230.899994  228.259995  228.610001   
2025-08-28  232.559998  232.559998  233.410004  229.339996  230.820007   
2025-08-29  232.139999  232.139999  233.380005  231.369995  232.509995   

Price         Volume  
Ticker          AAPL  
Date                  
2025-08-27  31259500  
2025-08-28  38074700  
2025-08-29  39389400  
Price        Adj Close       Close        High         Low        Open  \
Ticker            GOOG        GOOG        GOOG        GOOG        GOOG   
Date                                                                     
2025-08-27  208.210007  208.210007  209.570007  206.389999  206.460007   
2025-08-28  212.369995  212.369995  212.899994  207.600006  207.839996   
2025-08-29  213.529999  213.529999  215.339996 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price        Adj Close       Close        High         Low        Open  \
Ticker            MSFT        MSFT        MSFT        MSFT        MSFT   
Date                                                                     
2025-08-27  506.739990  506.739990  507.290009  499.899994  502.000000   
2025-08-28  509.640015  509.640015  511.089996  505.500000  507.089996   
2025-08-29  506.690002  506.690002  509.600006  504.489990  508.660004   

Price         Volume  
Ticker          MSFT  
Date                  
2025-08-27  17277900  
2025-08-28  18015600  
2025-08-29  20954200  


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AMZN,AMZN,AMZN,AMZN,AMZN,AMZN
Date,,,,,,
2025-08-27,229.119995,229.119995,229.869995,227.809998,228.570007,21254500
2025-08-28,231.600006,231.600006,232.710007,228.020004,229.009995,33679600
2025-08-29,229.000000,229.000000,231.809998,228.160004,231.320007,26177800


In [ ]:
tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN']
# tech_list = ['AAPL', 'GOOG']

end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)

company_list = []

for stock in tech_list:
    # globals()[stock] = yf.download(stock, start, end,auto_adjust=False)
    df = yf.download(stock, start, end, auto_adjust=False)
    df.head
    company_list.append(df)

company_name = ["APPLE", "GOOGLE", "MICROSOFT", "AMAZON"]

for company, com_name in zip(company_list, company_name):
    company["company_name"] = com_name

df = pd.concat(company_list, axis=0)
df.tail(10)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Price      Adj Close Close High  Low Open Volume company_name Adj Close Close  \
Ticker          AAPL  AAPL AAPL AAPL AAPL   AAPL                   GOOG  GOOG   
Date                                                                            
2025-08-18       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON       NaN   NaN   
2025-08-19       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON       NaN   NaN   
2025-08-20       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON       NaN   NaN   
2025-08-21       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON       NaN   NaN   
2025-08-22       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON       NaN   NaN   
2025-08-25       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON       NaN   NaN   
2025-08-26       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON       NaN   NaN   
2025-08-27       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON       NaN   NaN   
2025-08-28       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON       NaN   NaN   
2025-08-29       NaN   NaN  NaN  NaN  NaN    NaN       AMAZON       NaN   NaN   

Price      High  ...       Low Open Volume   Adj Close       Close  \
Ticker     GOOG  ... MSFT MSFT MSFT   MSFT        AMZN        AMZN   
Date             ...                                                 
2025-08-18  NaN  ...  NaN  NaN  NaN    NaN  231.490005  231.490005   
2025-08-19  NaN  ...  NaN  NaN  NaN    NaN  228.009995  228.009995   
2025-08-20  NaN  ...  NaN  NaN  NaN    NaN  223.809998  223.809998   
2025-08-21  NaN  ...  NaN  NaN  NaN    NaN  221.949997  221.949997   
2025-08-22  NaN  ...  NaN  NaN  NaN    NaN  228.839996  228.839996   
2025-08-25  NaN  ...  NaN  NaN  NaN    NaN  227.940002  227.940002   
2025-08-26  NaN  ...  NaN  NaN  NaN    NaN  228.710007  228.710007   
2025-08-27  NaN  ...  NaN  NaN  NaN    NaN  229.119995  229.119995   
2025-08-28  NaN  ...  NaN  NaN  NaN    NaN  231.600006  231.600006   
2025-08-29  NaN  ...  NaN  NaN  NaN    NaN  229.000000  229.000000   

Price             High         Low        Open      Volume  
Ticker            AMZN        AMZN        AMZN        AMZN  
Date                                                        
2025-08-18  231.910004  228.330002  230.229996  25248900.0  
2025-08-19  230.529999  227.119995  230.089996  29891000.0  
2025-08-20  227.270004  220.919998  227.119995  36604300.0  
2025-08-21  222.779999  220.500000  222.649994  32140500.0  
2025-08-22  229.139999  220.820007  222.789993  37315300.0  
2025-08-25  229.600006  227.309998  227.350006  22633700.0  
2025-08-26  229.000000  226.020004  227.110001  26105400.0  
2025-08-27  229.869995  227.809998  228.570007  21254500.0  
2025-08-28  232.710007  228.020004  229.009995  33679600.0  
2025-08-29  231.809998  228.160004  231.320007  26177800.0  

[10 rows x 25 columns]